### Dependencies

In [22]:
# Make a trueskill model using pyro
import numpy as np
import torch
import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
from scipy.special import expit as sigmoid
from tqdm import tqdm

# For plotting
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_dark"

### Model

In [23]:
perf_variance = 5.0

class TrueskillModel:
     
    def __init__(self, mus=None, sigmas=None):
        if mus is None:
            mus = []
        if sigmas is None:
            sigmas = []

        if len(mus) != len(sigmas):
            raise ValueError('mus and sigmas must have the same length')

        self.mus = torch.tensor(mus)
        self.sigmas = torch.tensor(sigmas)
        self.rankings = torch.clamp((self.mus - 3*self.sigmas).int(), min=0)
        

    def __getitem__(self, i):
        return self.mus[i], self.sigmas[i]

    def __len__(self):
        return len(self.mus)
    
    def __add__(self, other):
        return TrueskillModel(torch.cat((self.mus, other.mus)), torch.cat((self.sigmas, other.sigmas)))
    
    
    def add_player(self, mu, sigma):
        self.mus = torch.cat((self.mus, torch.tensor([mu])))
        self.sigmas = torch.cat((self.sigmas, torch.tensor([sigma])))
        self.rankings = torch.cat((self.rankings, torch.clamp((torch.tensor([mu - 3*sigma]).int()), min=0)))

    def infer_matches(self, matches:list):
        for match in matches:
            self.infer_match(match)

    def infer_match(self, match:dict):
        
        print('Infering match:', match['player1'], 'vs', match['player2'])

        # Data
        data = torch.tensor(match['outcome'])
        hyperparameters = {
            'mu1': self.mus[match['player1']],
            'sigma1': self.sigmas[match['player1']],
            'mu2': self.mus[match['player2']],
            'sigma2': self.sigmas[match['player2']]
        }
        
        # Optimizer
        adam_params = {"lr": 0.01}
        optimizer = Adam(adam_params)

        # SVI
        svi = SVI(self.model, self.guide, optimizer, loss=Trace_ELBO())

        # Inference
        for step in tqdm(range(1000)):
            loss = svi.step(data, hyperparameters)
            # if step % 100 == 0:
            #     print('step: {} loss: {}'.format(step, loss))
        
        # Update
        self.mus[match['player1']] = pyro.param('mu1').item()
        self.sigmas[match['player1']] = pyro.param('sigma1').item()
        self.mus[match['player2']] = pyro.param('mu2').item()
        self.sigmas[match['player2']] = pyro.param('sigma2').item()
        self.rankings[match['player1']] = torch.clamp((self.mus[match['player1']] - 3*self.sigmas[match['player1']]).int(), min=0)
        self.rankings[match['player2']] = torch.clamp((self.mus[match['player2']] - 3*self.sigmas[match['player2']]).int(), min=0)


    def model(self, data:torch.Tensor, hyperparameters:dict):

        # Hyperparameters
        mu1 = hyperparameters['mu1']
        sigma1 = hyperparameters['sigma1']
        mu2 = hyperparameters['mu2']
        sigma2 = hyperparameters['sigma2']

        # Skill
        s1 = pyro.sample('skill1', dist.Normal(mu1, sigma1))
        s2 = pyro.sample('skill2', dist.Normal(mu2, sigma2))

        # Performance
        p1 = pyro.sample('perf1', dist.Normal(s1, perf_variance))
        p2 = pyro.sample('perf2', dist.Normal(s2, perf_variance))

        # # Draw margin
        # m = pyro.sample('margin', dist.Normal(1, torch.sqrt(10)))

        # Outcome (1 if player 1 wins, 0 if player 2 wins)
        diff = p1 - p2
        return pyro.sample('outcome', dist.Bernoulli(logits=diff), obs=data.float())


    def guide(self, data:torch.Tensor, hyperparameters:dict):
        
        # Hyperparameters
        mu1 = pyro.param('mu1', hyperparameters['mu1'])
        sigma1 = pyro.param('sigma1', hyperparameters['sigma1'], constraint=dist.constraints.positive)
        mu2 = pyro.param('mu2', hyperparameters['mu2'])
        sigma2 = pyro.param('sigma2', hyperparameters['sigma2'], constraint=dist.constraints.positive)

        # Skill
        pyro.sample('skill1', dist.Normal(mu1, sigma1))
        pyro.sample('skill2', dist.Normal(mu2, sigma2))

        # Performance
        pyro.sample('perf1', dist.Normal(mu1, perf_variance))
        pyro.sample('perf2', dist.Normal(mu2, perf_variance))


### Define model

In [12]:
# mus = [108.4022, 124.4276,  87.8523, 122.9843, 106.9173, 110.6769,  74.3508, 75.0743,  94.3534,  90.7460, 287.1037]
# sigma = [1.4091e+00, 2.4500e-02, 1.5174e-10, 5.6699e+00, 9.6327e-01, 1.2125e-01, 5.1401e+00, 9.4271e-02, 1.6267e+00, 9.4192e-02, 6.4496e-04]



In [27]:
num_matches = 10
num_players = 2

# Define model
mus = [50., 300.]
sigmas = [40., 1.]
model = TrueskillModel(mus, sigmas)

# Make match where player 1 wins
match = {'player1': 0, 'player2': 1, 'outcome': 1}

# Preallocate arrays
M = np.zeros((num_players, num_matches+1))
S = np.zeros((num_players, num_matches+1))
R = np.zeros((num_players, num_matches+1))

# Save initial values
M[:, 0] = model.mus
S[:, 0] = model.sigmas
R[:, 0] = model.rankings

# Infer matches
for i in range(num_matches):
    model.infer_match(match)
    M[:, i+1] = model.mus
    S[:, i+1] = model.sigmas
    R[:, i+1] = model.rankings

Infering match: 0 vs 1


100%|██████████| 1000/1000 [00:08<00:00, 122.94it/s]


Infering match: 0 vs 1


100%|██████████| 1000/1000 [00:07<00:00, 137.95it/s]


Infering match: 0 vs 1


...

100%|██████████| 1000/1000 [00:07<00:00, 128.12it/s]


Infering match: 0 vs 1


100%|██████████| 1000/1000 [00:07<00:00, 137.57it/s]


Infering match: 0 vs 1


100%|██████████| 1000/1000 [00:07<00:00, 137.38it/s]


In [28]:
# Plot
fig = go.Figure()
trace1 = go.Scatter(x=np.arange(num_matches+1), y=M[0], mode='lines', name='Player 1')
trace2 = go.Scatter(x=np.arange(num_matches+1), y=M[1], mode='lines', name='Player 2')
fig.add_trace(trace1)
fig.add_trace(trace2)
fig.update_layout(title='Trueskill', xaxis_title='Match', yaxis_title='Skill')
fig.show()

### Run random matches

In [16]:
# Generate random matches for players
num_matches = 10
matches = []
for i in range(10):
    # Draw 2 random players that are not the same
    player1, player2 = np.random.choice(10, 2, replace=False)
    outcome = np.random.choice([0, 1])
    matches.append({'player1': player1, 'player2': player2, 'outcome': outcome})

print('Initial rankings:', model.rankings)
model.infer_matches(matches)
print('Final rankings:', model.rankings)


Initial rankings: tensor([ 50, 294], dtype=torch.int32)
Infering match: 9 vs 5


IndexError: index 9 is out of bounds for dimension 0 with size 2